# Links parameters in panel

In the  [param user guide](Param.ipynb), we have seen how parameterized classes are transformed to user interface at free cost. However in certain cases, we need more customisation over widgets automatically generated. Indeed widgets created with `widgets` modules exposed more attributes and allow more possibilities.

In this notebook we will see how to links custom panels between them:

 - links between `Parameters` of `Parameterized` classes (most of panel inherit from `param.Parameterized`) with `link` method
 - links between `Parameters` and custom functions with `watch` method

In [ ]:
import param
import panel as pn
pn.extension()

## Links between `params` of `Parameterized` classes

In this example we will link a markdown panel to a text input widget

In [ ]:
mkdown_pane = pn.pane.Markdown('Markdown display')
mkdown_pane

In [ ]:
widget_text = pn.widgets.TextInput(value=mkdown_pane.object)
widget_text

We verify both objects we want to link are `param.Parameterized` objects

In [ ]:
print(isinstance(mkdown_pane, param.Parameterized), isinstance(widget_text, param.Parameterized))

Link of widget `value` attribute to the `object` attribute of the panel

In [ ]:
widget_text.link(mkdown_pane, value = 'object')
widget_text.value = 'New text' #above displays should reflect change

Accessing to available parameters using `params` method

In [ ]:
widget_text.params()

## Link `params` with `watch` method 

Previously we have seen how to link one to one parameters between to Parameterized classes. However value between `parameters` must be the same. Sometimes parameters are linked but values differ. In this case we will add a `watcher`. A `watcher` is a function called when the attribute is changed.

Let's see how to link the disabled property of a `TextInput` widget to a `ToggleButton`

widget text creation

In [ ]:
widget_text = pn.widgets.TextInput(value='text', disabled=True)
widget_text

toggle button creation

In [ ]:
widget_toggle = pn.widgets.Toggle(active=False, name='Click to enable text')
widget_toggle

We could link toggle `active` parameter to the text widget `disabled` parameter with:
```python
widget_toggle.link(widget_text, active = 'disabled')
```
However we want the opposite behavior, when toggle is `active=True` text input is enabled (`disabled=False`)

To do it we use the `watch` method of the class `parameters` to connect the toggle button to the text input

In [ ]:
watcher = widget_toggle.param.watch(lambda change: setattr(widget_text, 'disabled', not change.new), 'active' )

Now cliking on the toggle button enable or disable the text input

`param.watch` return a reference to the watcher which can be used to remove it.

In [ ]:
widget_toggle.param.unwatch(watcher)

the two widgets are not linked anymore

## More advance usage

Using the `Sine` example of the [param user guide](Param.ipynb), we will create sliders with a custom step value. Indeed the default step of sliders automatically generated is 0.1 and is not always adapted.

In [ ]:
import numpy as np

class Sine(param.Parameterized):

    phase = param.Number(default=0, bounds=(0, np.pi))

    frequency = param.Number(default=1, bounds=(0.1, 2))
    
    
    def widgets(self):
        phase_params = self.params()['phase']
        frequency_params = self.params()['frequency']
        phase_slider = pn.widgets.FloatSlider(value=phase_params.default,
                                              start=phase_params.bounds[0],
                                              end=phase_params.bounds[1], step=1e-2)
        frequency_slider = pn.widgets.FloatSlider(value=frequency_params.default,
                                                  start=frequency_params.bounds[0],
                                                  end=frequency_params.bounds[1], step=1e-2)
        phase_slider.link(self, value='phase')
        frequency_slider.link(self, value='frequency')
        return pn.widgets.WidgetBox(frequency_slider, phase_slider)

    @param.depends('phase', 'frequency')
    def view(self):
        y = np.sin(np.linspace(0, np.pi*3, 40)*self.frequency+self.phase)
        y = ((y-y.min())/y.ptp())*20
        array = np.array([list((' '*(int(round(d))-1) + '*').ljust(20)) for d in y])
        return pn.pane.Str('\n'.join([''.join(r) for r in array.T]), height=325, width=500)

sine = Sine(name='ASCII Sine Wave')
pn.panel(pn.Row(sine.widgets(), sine.view))